In [4]:
!pip install pyautogen[websockets] fastapi uvicorn

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ---------------------------------------- 0.0/125.0 kB ? eta -:--:--
     --------- --------------------------- 30.7/125.0 kB 660.6 kB/s eta 0:00:01
     ------------------ ------------------ 61.4/125.0 kB 825.8 kB/s eta 0:00:01
     --------------------------- --------- 92.2/125.0 kB 655.4 kB/s eta 0:00:01
     ------------------------------------ 125.0/125.0 kB 735.5 kB/s eta 0:00:00


In [5]:
from datetime import datetime
from tempfile import TemporaryDirectory

from websockets.sync.client import connect as ws_connect

import autogen
from autogen.io.websockets import IOWebsockets

config_list = autogen.config_list_from_json(
    env_or_file="OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4o", "gpt-4", "gpt-3.5-turbo-16k"],
    },
)

当客户端连接到 websocket 服务器时，服务器会自动初始化 IOWebsockets 类的一个新实例。这个实例对于管理服务器和客户端之间数据流至关重要。 on_connect 函数利用这个实例来设置通信协议，定义交互规则，并执行任何必要的初步数据交换或配置，以确保客户端和服务器之间的交互顺利进行。

In [6]:
def on_connect(iostream: IOWebsockets) -> None:
    print(f" - on_connect(): Connected to client using IOWebsockets {iostream}", flush=True)

    print(" - on_connect(): Receiving message from client.", flush=True)

    # 1. Receive Initial Message
    initial_msg = iostream.input()

    # 2. Instantiate ConversableAgent
    agent = autogen.ConversableAgent(
        name="chatbot",
        system_message="Complete a task given to you and reply TERMINATE when the task is done. If asked about the weather, use tool 'weather_forecast(city)' to get the weather forecast for a city.",
        llm_config={
            "config_list": autogen.config_list_from_json(
                env_or_file="OAI_CONFIG_LIST"              
            ),
            "stream": True,
        },
    )

    # 3. Define UserProxyAgent
    user_proxy = autogen.UserProxyAgent(
        name="user_proxy",
        system_message="A proxy for the user.",
        is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
        human_input_mode="NEVER",
        max_consecutive_auto_reply=10,
        code_execution_config=False,
    )

    # 4. Define Agent-specific Functions
    def weather_forecast(city: str) -> str:
        return f"The weather forecast for {city} at {datetime.now()} is sunny."

    autogen.register_function(
        weather_forecast, caller=agent, executor=user_proxy, description="Weather forecast for a city"
    )

    # 5. Initiate conversation
    print(
        f" - on_connect(): Initiating chat with agent {agent} using message '{initial_msg}'",
        flush=True,
    )
    user_proxy.initiate_chat(  
        agent,
        message=initial_msg,
    )

使用 HTML/JS 客户端测试在 FastAPI 服务器内部运行的 WebSocket 服务器

In [10]:
from contextlib import asynccontextmanager  

from fastapi import FastAPI  
from fastapi.responses import HTMLResponse 

PORT = 8000

html = """
<!DOCTYPE html>
<html>
    <head>
        <title>Autogen websocket test</title>
    </head>
    <body>
        <h1>WebSocket Chat</h1>
        <form action="" onsubmit="sendMessage(event)">
            <input type="text" id="messageText" autocomplete="off"/>
            <button>Send</button>
        </form>
        <ul id='messages'>
        </ul>
        <script>
            var ws = new WebSocket("ws://127.0.0.1:8080/ws");
            ws.onmessage = function(event) {
                var messages = document.getElementById('messages')
                var message = document.createElement('li')
                var content = document.createTextNode(event.data)
                message.appendChild(content)
                messages.appendChild(message)
            };
            function sendMessage(event) {
                var input = document.getElementById("messageText")
                ws.send(input.value)
                input.value = ''
                event.preventDefault()
            }
        </script>
    </body>
</html>
"""


@asynccontextmanager
async def run_websocket_server(app):
    with IOWebsockets.run_server_in_thread(on_connect=on_connect, port=8080) as uri:
        print(f"Websocket server started at {uri}.", flush=True)

        yield


app = FastAPI(lifespan=run_websocket_server)


@app.get("/")
async def get():
    return HTMLResponse(html) 

In [11]:
import uvicorn 

config = uvicorn.Config(app, port=8081)
server = uvicorn.Server(config)
await server.serve()  

INFO:     Started server process [25952]
INFO:     Waiting for application startup.


Websocket server started at ws://127.0.0.1:8080.


INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8081 (Press CTRL+C to quit)


INFO:     127.0.0.1:64729 - "GET / HTTP/1.1" 200 OK
 - on_connect(): Connected to client using IOWebsockets <autogen.io.websockets.IOWebsockets object at 0x000001CE9F7B6ED0>
 - on_connect(): Receiving message from client.
 - on_connect(): Initiating chat with agent <autogen.agentchat.conversable_agent.ConversableAgent object at 0x000001CE9F7B3CE0> using message '你好'


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [25952]
